In [1]:
import pandas as pd

df = pd.read_csv('trucks.csv')
df

,seq,type,timestamp,truckId,positionLatitude,positionLongitude,equipType,nextTripLengthPreference
0,15,Truck,2023-11-18T08:12:28,101,34.294167,-85.879997,Reefer,Long
1,16,Truck,2023-11-18T08:19:59,134,39.913216,-86.210175,Flatbed,Short
2,17,Truck,2023-11-18T08:20:41,121,36.077549,-87.120148,Van,Short
3,19,Truck,2023-11-18T08:28:02,106,32.505447,-85.605026,Van,Long
4,20,Truck,2023-11-18T08:29:42,147,35.794121,-86.377892,Flatbed,Long
...,...,...,...,...,...,...,...,...
1995,2496,Truck,2023-11-18T22:58:03,238,42.132271,-72.051010,Reefer,Short
1996,2497,Truck,2023-11-18T22:59:24,353,42.473766,-79.276031,Flatbed,Short
1997,2498,Truck,2023-11-18T22:59:27,290,39.569466,-76.139107,Van,Short
1998,2499,Truck,2023-11-18T22:59:35,241,38.293682,-85.542076,Flatbed,Long


In [2]:
serie = df['truckId'].value_counts()
serie


366    19
106    14
292    14
110    14
331    14
       ..
123     1
400     1
197     1
219     1
222     1
Name: truckId, Length: 322, dtype: int64

In [3]:
df.loc[df['truckId'] == 163]

,seq,type,timestamp,truckId,positionLatitude,positionLongitude,equipType,nextTripLengthPreference
144,230,Truck,2023-11-18T10:13:14,163,39.345943,-84.611092,Van,Long
221,348,Truck,2023-11-18T10:53:18,163,39.058472,-84.469650,Van,Long
988,1450,Truck,2023-11-18T16:09:41,163,39.803234,-83.208191,Van,Long
1092,1575,Truck,2023-11-18T16:49:44,163,40.224995,-82.929070,Van,Long
1215,1709,Truck,2023-11-18T17:29:50,163,40.777355,-82.416084,Van,Long
1418,1912,Truck,2023-11-18T18:38:20,163,41.035187,-81.755264,Van,Long
1466,1961,Truck,2023-11-18T18:53:19,163,41.036072,-81.504890,Van,Long
1816,2316,Truck,2023-11-18T21:20:21,163,41.204025,-79.921600,Van,Long
1893,2393,Truck,2023-11-18T22:00:23,163,41.170616,-79.101173,Van,Long
1952,2452,Truck,2023-11-18T22:30:25,163,41.083035,-78.506371,Van,Long


In [4]:
df = pd.read_csv('notifications.csv')
df

,timestamp,price,estimated_profit,estimated_distance,estimated_wage,truck_id,load_id
0,2023-11-18T11:33:07,1045.00,425.988886,448.558778,61.729430,206,10713
1,2023-11-18T11:34:58,780.78,188.426192,429.241890,28.533335,230,10586
2,2023-11-18T11:34:58,1400.00,237.943967,842.069589,18.367078,230,10147
3,2023-11-18T11:35:06,1050.00,229.690715,594.427018,25.116450,206,10078
4,2023-11-18T11:35:06,1050.00,261.582495,571.317032,29.760818,246,10078
...,...,...,...,...,...,...,...
8113,2023-11-18T22:59:27,465.00,237.587203,164.791882,93.713162,290,10002
8114,2023-11-18T22:59:27,600.00,326.859221,197.928100,107.341248,290,10499
8115,2023-11-18T22:59:27,332.50,191.260079,102.347769,121.467281,290,10009
8116,2023-11-18T22:59:35,800.00,386.256064,299.814446,83.740609,241,11019


In [5]:
serie = df['truck_id'].value_counts()
serie



163    127
266    122
160    119
378    111
411    108
      ... 
399      1
112      1
221      1
244      1
257      1
Name: truck_id, Length: 233, dtype: int64

In [7]:
df.loc[df['truck_id'] == 163]

,timestamp,price,estimated_profit,estimated_distance,estimated_wage,truck_id,load_id
2950,2023-11-18T16:09:41,1200.00,216.805016,712.460133,19.779810,163,11057
2951,2023-11-18T16:09:41,1550.00,341.430350,875.775109,25.340949,163,10616
2952,2023-11-18T16:09:41,1500.00,223.014725,925.351648,15.665350,163,10630
2953,2023-11-18T16:09:41,913.00,173.093468,536.164154,20.984386,163,10336
2954,2023-11-18T16:09:41,1100.00,289.243386,587.504793,32.001135,163,10637
...,...,...,...,...,...,...,...
7892,2023-11-18T22:30:25,730.40,310.105971,304.560890,66.183442,163,10985
7893,2023-11-18T22:30:25,1000.00,429.610351,413.325833,67.560918,163,11059
7894,2023-11-18T22:30:25,1100.00,457.026404,465.922896,63.758868,163,10245
7895,2023-11-18T22:30:25,1300.00,568.167873,530.313136,69.639821,163,10466
